In [1]:
import torch_geometric
from topobenchmarkx.data.datasets import CustomDataset
import rootutils

rootutils.setup_root("./", indicator=".project-root", pythonpath=True)

import hydra
from hydra.experimental import initialize, compose

initialize(config_path="../configs", job_name="job")
config = compose(config_name="train.yaml", return_hydra_config=True)

/usr/local/lib/python3.11/site-packages/hydra/experimental/initialize.py:43: UserWarning: hydra.experimental.initialize() is no longer experimental. Use hydra.initialize()
  deprecation_warning(message=message)
/usr/local/lib/python3.11/site-packages/hydra/experimental/initialize.py:45: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  self.delegate = real_initialize(
/usr/local/lib/python3.11/site-packages/hydra/experimental/compose.py:25: UserWarning: hydra.experimental.compose() is no longer experimental. Use hydra.compose()
  deprecation_warning(message=message)


In [2]:
# # Load data
# from topobenchmarkx.data.load.loaders import HypergraphLoader

# data_loader = HypergraphLoader(config)
# data = data_loader.load()

In [11]:
data_loader = hydra.utils.instantiate(config.data)
data_lst = data_loader.load()
# dataset = CustomDataset(data_lst)
data = data_lst[0]
data.edge_index = data.edge_index.coalesce().indices()
data_lst = [data, data]

Loading hypergraph dataset name: Cora
number of nodes:2708, feature dimension: 1433
number of hyperedges: 1072
Final num_hyperedges 1392
Final num_nodes 2708
Final num_class 7


In [12]:
data

Data(x=[2708, 1433], edge_index=[2, 4905], y=[2708], n_x=2708, num_hyperedges=1392, num_class=7, train_mask=[1354], val_mask=[677], test_mask=[677])

In [14]:
databatch = torch_geometric.data.Batch.from_data_list(data_lst)

In [17]:
databatch.edge_index[:, :4905]

tensor([[   0,    1,    2,  ..., 2706, 2706, 2707],
        [ 978,  736,  399,  ...,  362,  419,  921]])

In [18]:
databatch.edge_index[:, 4905:]

tensor([[2708, 2709, 2710,  ..., 5414, 5414, 5415],
        [3686, 3444, 3107,  ..., 3070, 3127, 3629]])

In [22]:
databatch.edge_index = databatch.edge_index.to_sparse()
databatch.edge_index

tensor(indices=tensor([[   0,    0,    0,  ...,    1,    1,    1],
                       [   1,    2,    3,  ..., 9807, 9808, 9809]]),
       values=tensor([   1,    2,    2,  ..., 3070, 3127, 3629]),
       size=(2, 9810), nnz=19617, layout=torch.sparse_coo)

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader


class TextDataset(Dataset):
    def __init__(self, text_list, labels):
        self.text_list = text_list
        self.labels = labels

    def __len__(self):
        return len(self.text_list)

    def __getitem__(self, idx):
        text = self.text_list[idx]
        label = self.labels[idx]

        # You can perform any text preprocessing here if needed
        # For example, tokenization, numerical encoding, etc.

        return {"text": text, "label": label}


# Example usage
text_data = [
    "This is an example.",
    "Another text sample.",
    "PyTorch DataLoader with text.",
]
labels = [0, 1, 0]  # Example labels

# Create an instance of your custom dataset
custom_dataset = TextDataset(text_data, labels)

# Use DataLoader to load batches of data
batch_size = 2
data_loader = DataLoader(custom_dataset, batch_size=batch_size, shuffle=True)

# Iterate through batches
for batch in data_loader:
    texts = batch["text"]
    labels = batch["label"]

    # Perform your training/validation/test operations here
    print("Texts:", texts)
    print("Labels:", labels)

In [ ]:
data_lst[0].keys()

In [ ]:
from torch.utils.data import DataLoader

dataloader = DataLoader(dataset, batch_size=1, shuffle=True)
next(iter(dataloader))

In [ ]:
from torch_geometric.loader import DataLoader

dataloader = DataLoader(dataset, batch_size=1, shuffle=False)

next(iter(dataloader))

In [ ]:
config.data

### Loss

In [ ]:
loss = hydra.utils.instantiate(config.model.loss)

### Backbone

In [ ]:
hydra.utils.instantiate(config.model.backbone)

### ReadOut

In [ ]:
readout = hydra.utils.instantiate(config.model.readout)

In [ ]:
readout.parameters()

### Evaluator

In [ ]:
evaluator = hydra.utils.instantiate(config.evaluator)

In [ ]:
import torch

d = {
    "labels": torch.tensor([0, 1, 2, 1, 2, 2]),
    "logits": torch.tensor(
        [[1, 11, 11], [5, 11, 3], [2, 3, 4], [5, 16, 7], [8, 9, 10], [11, 12, 13]]
    ),
}


evaluator.eval(d)

In [ ]:
d["logits"].argmax(dim=1)

### Dataloader

In [ ]:
# Load data
from topobenchmarkx.data.load.loaders import HypergraphLoader
from topobenchmarkx.data.dataloader_fullbatch import FullBatchDataModule

data_loader = HypergraphLoader(config)
data = data_loader.load()
dataloader = FullBatchDataModule(data=data)

In [ ]:
batch = next(iter(dataloader.train_dataloader()))

In [ ]:
batch.x[batch.train_mask]

In [ ]:
a = hydra.utils.instantiate(config.model)

In [ ]:
a.hparams.backbone is a.backbone

In [ ]:
id(a.hparams.backbone) == id(a.backbone)

In [ ]:
b = a.backbone.__class__

In [ ]:
b in []topomodelx.nn.hypergraph.unigcnii.UniGCNII

In [ ]:
import topomodelx